In [1]:
import pandas as pd

In [2]:
egypt_control = pd.read_csv("../data/egypt_control/experiments.csv")
us_control = pd.read_csv("../data/us_control/experiments.csv")
clean_urls_csv = pd.read_csv("../data_collection/urls_to_test.txt", header=None)
clean_urls = list(clean_urls_csv[0])
us_broken_indices = range(6753,6793)
us_control_clean = us_control[(us_control.url.isin(clean_urls) & ~(us_control.experiment_id.isin(us_broken_indices)))]
egypt_control_clean = egypt_control[(egypt_control.url.isin(clean_urls))]
potential_dns_poisoning = egypt_control_clean[((egypt_control_clean.default_lookup_success == 0) & (egypt_control_clean.cloudflare_lookup_success == 1))]
potential_dns_poisoning_domains = potential_dns_poisoning.domain.unique()
broken_dns_us = us_control_clean[(us_control_clean.domain.isin(potential_dns_poisoning_domains) & (us_control_clean.default_lookup_success == 0))]
dns_poisoned = potential_dns_poisoning[~(potential_dns_poisoning.domain.isin(broken_dns_us.domain))]
dns_poisoned_domains = list(set(potential_dns_poisoning_domains) - set(broken_dns_us.domain))
potential_blocked = egypt_control_clean[(egypt_control_clean.page_load_success == 0)]
potential_rst = potential_blocked[(potential_blocked.page_load_status.str.startswith('net::ERR_CONNECTION_RESET'))]
empty_resp = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_EMPTY_RESPONSE')]
connection_refused = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_CONNECTION_REFUSED')]
address_unreachable = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_ADDRESS_UNREACHABLE')]
connection_closed = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_CONNECTION_CLOSED')]
navigation_timeout_exceeded = potential_blocked[potential_blocked.page_load_status.str.startswith('Navigation timeout')]
timeout = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_TIMED_OUT')]
cert_date_invalid = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_CERT_DATE_INVALID')]
too_many_redirects = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_TOO_MANY_REDIRECTS')]
aborted = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_ABORTED')]
cert_name_invalid = potential_blocked[potential_blocked.page_load_status.str.startswith('net::ERR_CERT_COMMON_NAME_INVALID')]
nonblocked_domains = egypt_control_clean[((egypt_control_clean.page_load_success == 1) & ~(egypt_control_clean.url.isin(potential_blocked.url)) & (egypt_control_clean.default_lookup_success == 1) & ~(egypt_control_clean.domain.isin(dns_poisoned_domains)))]


In [3]:
rst_urls = list(set(list(potential_rst.url) + list(connection_refused.url)))
dropped_urls = list(set(list(address_unreachable.url) + list(address_unreachable.url) + list(connection_closed.url) + list(timeout.url) + list(navigation_timeout_exceeded.url) + list(empty_resp.url))) 
dns_poisoned_urls = list(set(dns_poisoned.url))
blocked_urls = list(set(list(rst_urls) + list(dropped_urls) + list(dns_poisoned_urls)))

In [4]:
egypt_control_sample_nat = pd.read_csv("../data/egypt_control_sample_nat/experiments.csv")
egypt_control_sample_bridged = pd.read_csv("../data/egypt_control_sample_bridged/experiments.csv")
egypt_mans_control = pd.read_csv("../data/egypt_mans_control/experiments.csv")
egypt_cloudflare = pd.read_csv("../data/egypt_cloudflare_new/experiments.csv")
egypt_geneva_def = pd.read_csv("../data/egypt_geneva_def/experiments.csv")
egypt_mans_geneva_def = pd.read_csv("../data/egypt_mans_geneva_def/experiments.csv")
egypt_geneva_evoled = pd.read_csv("../data/egypt_geneva_evolved/experiments.csv")
egypt_mans_geneva_evolved = pd.read_csv("../data/egypt_mans_geneva_evolved/experiments.csv")
egypt_intang_4 = pd.read_csv("../data/egypt_intang_4/experiments.csv")
egypt_intang_def = pd.read_csv("../data/egypt_intang_4/experiments.csv")
egypt_nordvpn = pd.read_csv("../data/egypt_nordvpn/experiments.csv")
egypt_nordvpn_obfs_bridged = pd.read_csv("../data/egypt_nordvpn_obfs_bridged/experiments.csv")
egypt_vpn_std_nat = pd.read_csv("../data/egypt_vpn_std_nat/experiments.csv")
egypt_tor = pd.read_csv("../data/egypt_tor/experiments.csv")
egypt_torbrowser = pd.read_csv("../data/egypt_torbrowser/experiments.csv")



In [5]:
results = {}

In [7]:
egypt_control_sample_nat.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,251.000000,251.000000,251.000000,2.510000e+02,251.0,251.000000,251.000000,251.000000
mean,125.000000,0.270916,0.505976,1.616671e+12,-120.0,0.848606,0.996016,0.262948
std,72.601653,0.445321,0.766136,1.265380e+06,0.0,0.359149,0.063119,0.441114
min,0.000000,0.000000,0.000000,1.616668e+12,-120.0,0.000000,0.000000,0.000000
25%,62.500000,0.000000,0.000000,1.616670e+12,-120.0,1.000000,1.000000,0.000000
50%,125.000000,0.000000,0.000000,1.616671e+12,-120.0,1.000000,1.000000,0.000000
75%,187.500000,1.000000,1.000000,1.616672e+12,-120.0,1.000000,1.000000,1.000000
max,250.000000,1.000000,4.000000,1.616674e+12,-120.0,1.000000,1.000000,1.000000


In [8]:
egypt_control_sample_nat[egypt_control_sample_nat.url.isin(blocked_urls)].describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,211.000000,211.000000,211.000000,2.110000e+02,211.0,211.000000,211.000000,211.000000
mean,123.810427,0.279621,0.559242,1.616671e+12,-120.0,0.824645,0.995261,0.137441
std,73.473883,0.449880,0.804776,1.278672e+06,0.0,0.381175,0.068843,0.345131
min,0.000000,0.000000,0.000000,1.616668e+12,-120.0,0.000000,0.000000,0.000000
25%,60.000000,0.000000,0.000000,1.616669e+12,-120.0,1.000000,1.000000,0.000000
50%,120.000000,0.000000,0.000000,1.616670e+12,-120.0,1.000000,1.000000,0.000000
75%,190.500000,1.000000,1.000000,1.616672e+12,-120.0,1.000000,1.000000,0.000000
max,250.000000,1.000000,4.000000,1.616674e+12,-120.0,1.000000,1.000000,1.000000


In [9]:
# sorted(list(egypt_control_sample_nat.page_load_status.unique()))

In [6]:
egypt_control_sample_nat["conn_rst"] = (egypt_control_sample_nat["page_load_status"].str.startswith('net::ERR_CONNECTION_RESET')).astype(int)
egypt_control_sample_nat["name_not_resolved"] = (egypt_control_sample_nat["page_load_status"].str.startswith('net::ERR_NAME_NOT_RESOLVED')).astype(int)
egypt_control_sample_nat["address_unreachable"] = (egypt_control_sample_nat["page_load_status"].str.startswith('net::ERR_ADDRESS_UNREACHABLE')).astype(int)
egypt_control_sample_nat["timed_out"] = (egypt_control_sample_nat["page_load_status"].str.startswith('net::ERR_TIMED_OUT')).astype(int)
egypt_control_sample_nat["navigation_timeout"] = (egypt_control_sample_nat["page_load_status"].str.startswith('Navigation timeout')).astype(int)
egypt_control_sample_nat["connection_closed"] = (egypt_control_sample_nat["page_load_status"].str.startswith('net::ERR_CONNECTION_CLOSED')).astype(int)
egypt_control_sample_nat["connection_refused"] = (egypt_control_sample_nat["page_load_status"].str.startswith('net::ERR_CONNECTION_REFUSED')).astype(int)
egypt_control_sample_nat["rst"] = (egypt_control_sample_nat["conn_rst"] | egypt_control_sample_nat["connection_refused"])
egypt_control_sample_nat["dropped"] = (egypt_control_sample_nat["address_unreachable"] | egypt_control_sample_nat["connection_closed"] | egypt_control_sample_nat["timed_out"] | egypt_control_sample_nat["navigation_timeout"])
egypt_control_sample_nat["dns_poison"] = (egypt_control_sample_nat["name_not_resolved"])
egypt_control_sample_nat.describe()


,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success,conn_rst,name_not_resolved,address_unreachable,timed_out,navigation_timeout,connection_closed,connection_refused,rst,dropped,dns_poison
count,251.000000,251.000000,251.000000,2.510000e+02,251.0,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000
mean,125.000000,0.270916,0.505976,1.616671e+12,-120.0,0.848606,0.996016,0.262948,0.430279,0.159363,0.019920,0.075697,0.019920,0.003984,0.019920,0.450199,0.119522,0.159363
std,72.601653,0.445321,0.766136,1.265380e+06,0.0,0.359149,0.063119,0.441114,0.496104,0.366745,0.140006,0.265042,0.140006,0.063119,0.140006,0.498508,0.325050,0.366745
min,0.000000,0.000000,0.000000,1.616668e+12,-120.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,62.500000,0.000000,0.000000,1.616670e+12,-120.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,125.000000,0.000000,0.000000,1.616671e+12,-120.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,187.500000,1.000000,1.000000,1.616672e+12,-120.0,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,250.000000,1.000000,4.000000,1.616674e+12,-120.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
results["control_nat"] = {
    "overall_success": egypt_control_sample_nat[egypt_control_sample_nat.url.isin(blocked_urls)].page_load_success.mean(),
    "rst_pct": egypt_control_sample_nat.rst.mean(),
    "drop_pct": egypt_control_sample_nat.dropped.mean(),
    "dns_poison_pct": egypt_control_sample_nat.dns_poison.mean(),
    "blocked_count": egypt_control_sample_nat[egypt_control_sample_nat.url.isin(blocked_urls)].page_load_success.count(),
    "dns_success": egypt_control_sample_nat[egypt_control_sample_nat.url.isin(dns_poisoned_urls)].page_load_success.mean(),
    "dns_poisoned_count": egypt_control_sample_nat[egypt_control_sample_nat.url.isin(dns_poisoned_urls)].page_load_success.count(),
    "rst_success": egypt_control_sample_nat[egypt_control_sample_nat.url.isin(rst_urls)].page_load_success.mean(),
    "rst_count": egypt_control_sample_nat[egypt_control_sample_nat.url.isin(rst_urls)].page_load_success.count(),
    "drop_success": egypt_control_sample_nat[egypt_control_sample_nat.url.isin(dropped_urls)].page_load_success.mean(),
    "drop_count": egypt_control_sample_nat[egypt_control_sample_nat.url.isin(dropped_urls)].page_load_success.count()   
}
results["control_nat"]

{'overall_success': 0.13744075829383887,
 'rst_pct': 0.450199203187251,
 'drop_pct': 0.11952191235059761,
 'dns_poison_pct': 0.1593625498007968,
 'blocked_count': 211,
 'dns_success': 0.14634146341463414,
 'dns_poisoned_count': 41,
 'rst_success': 0.2,
 'rst_count': 70,
 'drop_success': 0.0948905109489051,
 'drop_count': 137}

In [12]:
egypt_control_sample_bridged.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,183.000000,183.0,183.000000,1.830000e+02,183.0,183.000000,183.000000,183.000000
mean,91.000000,0.0,0.163934,1.616723e+12,-120.0,0.863388,0.994536,0.256831
std,52.971691,0.0,0.426344,7.882967e+05,0.0,0.344380,0.073922,0.438084
min,0.000000,0.0,0.000000,1.616721e+12,-120.0,0.000000,0.000000,0.000000
25%,45.500000,0.0,0.000000,1.616722e+12,-120.0,1.000000,1.000000,0.000000
50%,91.000000,0.0,0.000000,1.616723e+12,-120.0,1.000000,1.000000,0.000000
75%,136.500000,0.0,0.000000,1.616723e+12,-120.0,1.000000,1.000000,1.000000
max,182.000000,0.0,2.000000,1.616724e+12,-120.0,1.000000,1.000000,1.000000


In [13]:
egypt_control_sample_bridged["conn_rst"] = (egypt_control_sample_bridged["page_load_status"].str.startswith('net::ERR_CONNECTION_RESET')).astype(int)
egypt_control_sample_bridged["name_not_resolved"] = (egypt_control_sample_bridged["page_load_status"].str.startswith('net::ERR_NAME_NOT_RESOLVED')).astype(int)
egypt_control_sample_bridged["address_unreachable"] = (egypt_control_sample_bridged["page_load_status"].str.startswith('net::ERR_ADDRESS_UNREACHABLE')).astype(int)
egypt_control_sample_bridged["timed_out"] = (egypt_control_sample_bridged["page_load_status"].str.startswith('net::ERR_TIMED_OUT')).astype(int)
egypt_control_sample_bridged["navigation_timeout"] = (egypt_control_sample_bridged["page_load_status"].str.startswith('Navigation timeout')).astype(int)
egypt_control_sample_bridged["connection_closed"] = (egypt_control_sample_bridged["page_load_status"].str.startswith('net::ERR_CONNECTION_CLOSED')).astype(int)
egypt_control_sample_bridged["connection_refused"] = (egypt_control_sample_bridged["page_load_status"].str.startswith('net::ERR_CONNECTION_REFUSED')).astype(int)
egypt_control_sample_bridged["rst"] = (egypt_control_sample_bridged["conn_rst"] | egypt_control_sample_bridged["connection_refused"])
egypt_control_sample_bridged["dropped"] = (egypt_control_sample_bridged["address_unreachable"] | egypt_control_sample_bridged["connection_closed"] | egypt_control_sample_bridged["timed_out"] | egypt_control_sample_bridged["navigation_timeout"])
egypt_control_sample_bridged["dns_poison"] = (egypt_control_sample_bridged["name_not_resolved"])
egypt_control_sample_bridged.describe()


,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success,conn_rst,name_not_resolved,address_unreachable,timed_out,navigation_timeout,connection_closed,connection_refused,rst,dropped,dns_poison
count,183.000000,183.0,183.000000,1.830000e+02,183.0,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000
mean,91.000000,0.0,0.163934,1.616723e+12,-120.0,0.863388,0.994536,0.256831,0.475410,0.136612,0.016393,0.060109,0.021858,0.010929,0.021858,0.497268,0.109290,0.136612
std,52.971691,0.0,0.426344,7.882967e+05,0.0,0.344380,0.073922,0.438084,0.500765,0.344380,0.127331,0.238341,0.146621,0.104254,0.146621,0.501364,0.312858,0.344380
min,0.000000,0.0,0.000000,1.616721e+12,-120.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,45.500000,0.0,0.000000,1.616722e+12,-120.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,91.000000,0.0,0.000000,1.616723e+12,-120.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,136.500000,0.0,0.000000,1.616723e+12,-120.0,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,182.000000,0.0,2.000000,1.616724e+12,-120.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
results["control_bridged"] = {
    "overall_success": egypt_control_sample_bridged[egypt_control_sample_bridged.url.isin(potential_blocked.url)].page_load_success.mean(),
    "rst_pct": egypt_control_sample_bridged.rst.mean(),
    "drop_pct": egypt_control_sample_bridged.dropped.mean(),
    "dns_poison_pct": egypt_control_sample_bridged.dns_poison.mean(),
    "blocked_count": egypt_control_sample_bridged[egypt_control_sample_bridged.url.isin(blocked_urls)].page_load_success.count(),
    "dns_success": egypt_control_sample_bridged[egypt_control_sample_bridged.url.isin(dns_poisoned_urls)].page_load_success.mean(),
    "dns_poisoned_count": egypt_control_sample_bridged[egypt_control_sample_bridged.url.isin(dns_poisoned_urls)].page_load_success.count(),
    "rst_success": egypt_control_sample_bridged[egypt_control_sample_bridged.url.isin(rst_urls)].page_load_success.mean(),
    "rst_count": egypt_control_sample_bridged[egypt_control_sample_bridged.url.isin(rst_urls)].page_load_success.count(),
    "drop_success": egypt_control_sample_bridged[egypt_control_sample_bridged.url.isin(dropped_urls)].page_load_success.mean(),
    "drop_count": egypt_control_sample_bridged[egypt_control_sample_bridged.url.isin(dropped_urls)].page_load_success.count()   
}
results["control_bridged"]

{'overall_success': 0.12418300653594772,
 'rst_pct': 0.4972677595628415,
 'drop_pct': 0.1092896174863388,
 'dns_poison_pct': 0.1366120218579235,
 'blocked_count': 152,
 'dns_success': 0.1111111111111111,
 'dns_poisoned_count': 27,
 'rst_success': 0.14285714285714285,
 'rst_count': 56,
 'drop_success': 0.08080808080808081,
 'drop_count': 99}

In [15]:
# egypt_geneva_def.page_load_status.unique()

In [10]:
results = {}
policies = ["control_sample_bridged", 
# "mans_control", 
"geneva_def", 
# "mans_geneva_def", "mans_geneva_evolved", 
"intang_4", "intang_def", "nordvpn_obfs_bridged", "vpn_std_nat", "tor", "torbrowser", "geneva+cloudflare", "cloudflare_new",
# "control_sample_nat"
]
labels = {
    "control_sample_bridged": "Control",
    "geneva_def": "Geneva",
    "intang_4": "INTANG Policy 4",
    "nordvpn_obfs_bridged": "NordVPN Obfuscated",
    "vpn_std_nat": "Hotspot Shield",
    "tor": "Tor",
    "torbrowser": "Tor Browser",
    "geneva+cloudflare": "Geneva + Cloudflare",
    "cloudflare_new": "Cloudflare"

}
for policy in policies:
    print(policy)
    df = pd.read_csv(f'../data/egypt_{policy}/experiments.csv')
    df = df[~(df.page_load_status.str.startswith('net::ERR_SOCKS_CONNECTION_FAILED'))]
    df["conn_rst"] = (df["page_load_status"].str.startswith('net::ERR_CONNECTION_RESET'))
    df["name_not_resolved"] = (df["page_load_status"].str.startswith('net::ERR_NAME_NOT_RESOLVED'))
    df["address_unreachable"] = (df["page_load_status"].str.startswith('net::ERR_ADDRESS_UNREACHABLE'))
    df["timed_out"] = (df["page_load_status"].str.startswith('net::ERR_TIMED_OUT'))
    df["navigation_timeout"] = (df["page_load_status"].str.startswith('Navigation timeout') | df["page_load_status"].str.startswith('Timed out') | df['page_load_status'].str.startswith("Navigation failed"))
    df["connection_closed"] = (df["page_load_status"].str.startswith('net::ERR_CONNECTION_CLOSED'))
    df["empty_response"] = df["page_load_status"].str.startswith('net::ERR_EMPTY_RESPONSE')
    df["connection_refused"] = (df["page_load_status"].str.startswith('net::ERR_CONNECTION_REFUSED'))
    df["too_many_redirects"] = (df["page_load_status"].str.startswith('net::ERR_TOO_MANY_REDIRECTS'))
    df["rst"] = (df["conn_rst"] | df["connection_refused"]).astype(int)
    df["dropped"] = (df["address_unreachable"] | df["connection_closed"] | df["timed_out"] | df["navigation_timeout"] | df['too_many_redirects']  | df["empty_response"]).astype(int)
    df["dns_poison"] = (df["name_not_resolved"]).astype(int)
    results[policy] = {
        "overall_success": df[df.url.isin(blocked_urls)].page_load_success.mean() * 100,
        "rst_pct": df.rst.mean() * 100,
        "drop_pct": df.dropped.mean() * 100,
        "dns_poison_pct": df.dns_poison.mean() * 100,
        "success_pct": df.page_load_success.mean() * 100,
        "total_pct": df.rst.mean() + df.dropped.mean() + df.dns_poison.mean() + df.page_load_success.mean(),
        "blocked_count": df[df.url.isin(blocked_urls)].page_load_success.count(),
        "success_in_original_dns": df[df.url.isin(dns_poisoned_urls)].page_load_success.mean(),
        "original_dns_count": df[df.url.isin(dns_poisoned_urls)].page_load_success.count(),
        "success_in_original_rst": df[df.url.isin(rst_urls)].page_load_success.mean(),
        "original_rst_count": df[df.url.isin(rst_urls)].page_load_success.count(),
        "success_in_original_drop": df[df.url.isin(dropped_urls)].page_load_success.mean(),
        "original_drop_count": df[df.url.isin(dropped_urls)].page_load_success.count(),
        "rst_success": 1 - df[df.url.isin(rst_urls)].rst.mean(),
        "dns_success": 1 - df[df.url.isin(dns_poisoned_urls)].dns_poison.mean(),
        "drop_success": 1 - df[df.url.isin(dropped_urls)].dropped.mean()
    }
    # results[policy]['rst_success'] = 1 - results[policy]["rst_pct"]
    # results[policy]['dns_success'] = 1 - results[policy]["dns_poison_pct"]
    # results[policy]['drop_success'] = 1 - results[policy]["drop_pct"]
results_data = pd.DataFrame.from_dict(results, orient='index')

control_sample_nat
control_sample_bridged
mans_control
geneva_def
mans_geneva_def
mans_geneva_evolved
intang_4
intang_def
nordvpn_obfs_bridged
vpn_std_nat
tor
torbrowser
geneva+cloudflare
cloudflare_new


In [11]:
results_data.sort_values('overall_success', ascending=False)

,overall_success,rst_pct,drop_pct,dns_poison_pct,success_pct,total_pct,blocked_count,success_in_original_dns,original_dns_count,success_in_original_rst,original_rst_count,success_in_original_drop,original_drop_count,rst_success,dns_success,drop_success
tor,94.594595,0.000000,5.586592,0.000000,94.413408,1.0,148,0.925926,27,0.924528,53,0.969388,98,1.000000,1.000000,0.969388
torbrowser,93.288591,0.000000,5.555556,0.000000,94.444444,1.0,149,0.962963,27,0.943396,53,0.929293,99,1.000000,1.000000,0.929293
vpn_std_nat,92.105263,2.185792,2.732240,2.185792,92.896175,1.0,152,1.000000,27,0.875000,56,0.939394,99,0.946429,1.000000,0.969697
nordvpn_obfs_bridged,90.131579,1.092896,6.010929,1.639344,91.256831,1.0,152,0.962963,27,0.875000,56,0.919192,99,0.964286,0.962963,0.929293
geneva+cloudflare,84.868421,2.185792,10.382514,0.546448,86.885246,1.0,152,0.740741,27,0.803571,56,0.898990,99,0.946429,1.000000,0.909091
geneva_def,71.710526,1.639344,8.743169,13.114754,76.502732,1.0,152,0.148148,27,0.785714,56,0.878788,99,0.964286,0.148148,0.888889
mans_geneva_def,58.898305,1.779359,33.451957,2.846975,61.921708,1.0,236,0.551020,49,0.662338,77,0.600000,150,0.961039,0.959184,0.620000
mans_control,14.473684,59.562842,11.475410,1.092896,27.868852,1.0,152,0.259259,27,0.071429,56,0.121212,99,0.178571,1.000000,0.848485
control_sample_nat,13.744076,45.019920,12.749004,15.936255,26.294821,1.0,211,0.146341,41,0.200000,70,0.094891,137,0.271429,0.170732,0.781022
cloudflare_new,13.157895,57.377049,14.754098,0.546448,27.322404,1.0,152,0.259259,27,0.089286,56,0.101010,99,0.267857,1.000000,0.818182


In [15]:
egypt_mans_control.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,183.000000,183.0,183.000000,1.830000e+02,183.0,183.000000,183.000000,183.000000
mean,91.000000,0.0,0.163934,1.616813e+12,420.0,0.994536,0.989071,0.278689
std,52.971691,0.0,0.426344,8.613968e+05,0.0,0.073922,0.104254,0.449584
min,0.000000,0.0,0.000000,1.616811e+12,420.0,0.000000,0.000000,0.000000
25%,45.500000,0.0,0.000000,1.616812e+12,420.0,1.000000,1.000000,0.000000
50%,91.000000,0.0,0.000000,1.616813e+12,420.0,1.000000,1.000000,0.000000
75%,136.500000,0.0,0.000000,1.616813e+12,420.0,1.000000,1.000000,1.000000
max,182.000000,0.0,2.000000,1.616814e+12,420.0,1.000000,1.000000,1.000000


In [25]:
egypt_mans_control.cloudflare_lookup_success
egypt_mans_control[(egypt_mans_control.cloudflare_lookup_success == 0)]

,experiment_id,run_number,policy,domain,url,experiment_count,experiment_code,timestamp,timezone_offset,local_time,default_dns_servers,default_lookup_status,default_lookup_success,default_lookup_ips,cloudflare_lookup_status,cloudflare_lookup_success,cloudflare_lookup_ips,all_ips,page_load_success,page_load_status
52,52,0,egypt_mans_control,alalam.ir,http://www.alalam.ir,0,alalam.ir_0,1616811999922,420,"3/26/2021, 7:26:39 PM","[""127.0.0.53""]",success,1,"[""142.54.186.50""]",queryA ENODATA alalam.ir,0,[],"[""142.54.186.50""]",0,net::ERR_CONNECTION_RESET at http://www.alalam.ir
81,81,0,egypt_mans_control,jatoeg.org,http://www.jatoeg.org,0,jatoeg.org_0,1616812445255,420,"3/26/2021, 7:34:05 PM","[""127.0.0.53""]",queryA ENOTFOUND jatoeg.org,0,[],queryA ENOTFOUND jatoeg.org,0,[],[],0,net::ERR_NAME_NOT_RESOLVED at http://www.jatoe...


In [20]:
egypt_geneva_def.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,183.000000,183.0,183.000000,1.830000e+02,183.0,183.000000,183.000000,183.000000
mean,91.000000,0.0,0.163934,1.616884e+12,420.0,0.868852,0.994536,0.765027
std,52.971691,0.0,0.426344,1.061284e+06,0.0,0.338488,0.073922,0.425145
min,0.000000,0.0,0.000000,1.616882e+12,420.0,0.000000,0.000000,0.000000
25%,45.500000,0.0,0.000000,1.616883e+12,420.0,1.000000,1.000000,1.000000
50%,91.000000,0.0,0.000000,1.616884e+12,420.0,1.000000,1.000000,1.000000
75%,136.500000,0.0,0.000000,1.616885e+12,420.0,1.000000,1.000000,1.000000
max,182.000000,0.0,2.000000,1.616886e+12,420.0,1.000000,1.000000,1.000000


In [38]:
egypt_mans_geneva_def.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,281.000000,281.000000,281.000000,2.810000e+02,281.0,281.000000,281.000000,281.000000
mean,140.000000,2.085409,0.718861,1.616858e+12,420.0,0.978648,0.978648,0.619217
std,81.261922,1.333564,1.026071,2.838348e+07,0.0,0.144814,0.144814,0.486446
min,0.000000,0.000000,0.000000,1.616816e+12,420.0,0.000000,0.000000,0.000000
25%,70.000000,0.000000,0.000000,1.616820e+12,420.0,1.000000,1.000000,0.000000
50%,140.000000,3.000000,0.000000,1.616878e+12,420.0,1.000000,1.000000,1.000000
75%,210.000000,3.000000,1.000000,1.616879e+12,420.0,1.000000,1.000000,1.000000
max,280.000000,3.000000,6.000000,1.616881e+12,420.0,1.000000,1.000000,1.000000


In [46]:
egypt_mans_geneva_def[egypt_mans_geneva_def.run_number == 0].describe()
egypt_mans_geneva_def[egypt_mans_geneva_def.run_number == 3].describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,183.000000,183.0,183.000000,1.830000e+02,183.0,183.000000,183.000000,183.000000
mean,189.000000,3.0,0.912568,1.616879e+12,420.0,0.994536,0.994536,0.830601
std,52.971691,0.0,1.140178,1.080779e+06,0.0,0.073922,0.073922,0.376133
min,98.000000,3.0,0.000000,1.616877e+12,420.0,0.000000,0.000000,0.000000
25%,143.500000,3.0,0.000000,1.616878e+12,420.0,1.000000,1.000000,1.000000
50%,189.000000,3.0,1.000000,1.616879e+12,420.0,1.000000,1.000000,1.000000
75%,234.500000,3.0,1.000000,1.616880e+12,420.0,1.000000,1.000000,1.000000
max,280.000000,3.0,6.000000,1.616881e+12,420.0,1.000000,1.000000,1.000000


In [49]:
egypt_mans_geneva_evolved.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,124.000000,124.000000,124.000000,1.240000e+02,124.0,124.000000,124.000000,124.0
mean,61.500000,0.975806,0.153226,1.616890e+12,420.0,0.983871,0.991935,0.0
std,35.939764,0.154273,0.383488,2.756516e+06,0.0,0.126483,0.089803,0.0
min,0.000000,0.000000,0.000000,1.616885e+12,420.0,0.000000,0.000000,0.0
25%,30.750000,1.000000,0.000000,1.616888e+12,420.0,1.000000,1.000000,0.0
50%,61.500000,1.000000,0.000000,1.616890e+12,420.0,1.000000,1.000000,0.0
75%,92.250000,1.000000,0.000000,1.616893e+12,420.0,1.000000,1.000000,0.0
max,123.000000,1.000000,2.000000,1.616895e+12,420.0,1.000000,1.000000,0.0


In [50]:
egypt_intang_4.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,183.000000,183.0,183.000000,1.830000e+02,183.0,183.000000,183.000000,183.000000
mean,91.000000,0.0,0.163934,1.616888e+12,420.0,0.868852,0.994536,0.213115
std,52.971691,0.0,0.426344,9.632141e+05,0.0,0.338488,0.073922,0.410632
min,0.000000,0.0,0.000000,1.616886e+12,420.0,0.000000,0.000000,0.000000
25%,45.500000,0.0,0.000000,1.616887e+12,420.0,1.000000,1.000000,0.000000
50%,91.000000,0.0,0.000000,1.616888e+12,420.0,1.000000,1.000000,0.000000
75%,136.500000,0.0,0.000000,1.616889e+12,420.0,1.000000,1.000000,0.000000
max,182.000000,0.0,2.000000,1.616889e+12,420.0,1.000000,1.000000,1.000000


In [51]:
egypt_intang_def.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,183.000000,183.0,183.000000,1.830000e+02,183.0,183.000000,183.000000,183.000000
mean,91.000000,0.0,0.163934,1.616888e+12,420.0,0.868852,0.994536,0.213115
std,52.971691,0.0,0.426344,9.632141e+05,0.0,0.338488,0.073922,0.410632
min,0.000000,0.0,0.000000,1.616886e+12,420.0,0.000000,0.000000,0.000000
25%,45.500000,0.0,0.000000,1.616887e+12,420.0,1.000000,1.000000,0.000000
50%,91.000000,0.0,0.000000,1.616888e+12,420.0,1.000000,1.000000,0.000000
75%,136.500000,0.0,0.000000,1.616889e+12,420.0,1.000000,1.000000,0.000000
max,182.000000,0.0,2.000000,1.616889e+12,420.0,1.000000,1.000000,1.000000


In [53]:
egypt_nordvpn_obfs_bridged.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,183.000000,183.0,183.000000,1.830000e+02,183.0,183.000000,183.000000,183.000000
mean,91.000000,0.0,0.163934,1.616774e+12,-120.0,0.972678,0.978142,0.912568
std,52.971691,0.0,0.426344,1.004823e+06,0.0,0.163468,0.146621,0.283242
min,0.000000,0.0,0.000000,1.616772e+12,-120.0,0.000000,0.000000,0.000000
25%,45.500000,0.0,0.000000,1.616773e+12,-120.0,1.000000,1.000000,1.000000
50%,91.000000,0.0,0.000000,1.616774e+12,-120.0,1.000000,1.000000,1.000000
75%,136.500000,0.0,0.000000,1.616775e+12,-120.0,1.000000,1.000000,1.000000
max,182.000000,0.0,2.000000,1.616776e+12,-120.0,1.000000,1.000000,1.000000


In [54]:
egypt_vpn_std_nat.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,183.000000,183.0,183.000000,1.830000e+02,183.0,183.000000,183.000000,183.000000
mean,91.000000,0.0,0.163934,1.616780e+12,-120.0,0.978142,0.983607,0.928962
std,52.971691,0.0,0.426344,8.820006e+05,0.0,0.146621,0.127331,0.257593
min,0.000000,0.0,0.000000,1.616778e+12,-120.0,0.000000,0.000000,0.000000
25%,45.500000,0.0,0.000000,1.616779e+12,-120.0,1.000000,1.000000,1.000000
50%,91.000000,0.0,0.000000,1.616780e+12,-120.0,1.000000,1.000000,1.000000
75%,136.500000,0.0,0.000000,1.616780e+12,-120.0,1.000000,1.000000,1.000000
max,182.000000,0.0,2.000000,1.616781e+12,-120.0,1.000000,1.000000,1.000000


In [55]:
egypt_tor.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,183.000000,183.0,183.000000,1.830000e+02,183.0,183.000000,183.000000,183.000000
mean,91.000000,0.0,0.163934,1.616788e+12,-120.0,0.857923,0.994536,0.923497
std,52.971691,0.0,0.426344,1.179544e+06,0.0,0.350086,0.073922,0.266530
min,0.000000,0.0,0.000000,1.616785e+12,-120.0,0.000000,0.000000,0.000000
25%,45.500000,0.0,0.000000,1.616787e+12,-120.0,1.000000,1.000000,1.000000
50%,91.000000,0.0,0.000000,1.616788e+12,-120.0,1.000000,1.000000,1.000000
75%,136.500000,0.0,0.000000,1.616789e+12,-120.0,1.000000,1.000000,1.000000
max,182.000000,0.0,2.000000,1.616790e+12,-120.0,1.000000,1.000000,1.000000


In [56]:
egypt_torbrowser.describe()

,experiment_id,run_number,experiment_count,timestamp,timezone_offset,default_lookup_success,cloudflare_lookup_success,page_load_success
count,183.000000,183.0,183.000000,1.830000e+02,183.0,183.000000,183.000000,183.000000
mean,91.000000,0.0,0.163934,1.616802e+12,-120.0,0.863388,0.972678,0.928962
std,52.971691,0.0,0.426344,1.497113e+06,0.0,0.344380,0.163468,0.257593
min,0.000000,0.0,0.000000,1.616799e+12,-120.0,0.000000,0.000000,0.000000
25%,45.500000,0.0,0.000000,1.616801e+12,-120.0,1.000000,1.000000,1.000000
50%,91.000000,0.0,0.000000,1.616802e+12,-120.0,1.000000,1.000000,1.000000
75%,136.500000,0.0,0.000000,1.616803e+12,-120.0,1.000000,1.000000,1.000000
max,182.000000,0.0,2.000000,1.616804e+12,-120.0,1.000000,1.000000,1.000000
